**Note**  
**Training Notebook cannot run at kaggle platform (because many memory is requied to run).**  
**If you want to execute this code, you need to prepare own computations (out of kaggle).**  

# Baseline notebooks:
- Preprocessing : https://www.kaggle.com/code/motono0223/js24-preprocessing-create-lags
- Training (Code only) : **this notebook** https://www.kaggle.com/code/motono0223/js24-train-gbdt-model-with-lags-singlemodel
  - trained model : https://www.kaggle.com/datasets/motono0223/js24-trained-gbdt-model
- Inference : https://www.kaggle.com/code/motono0223/js24-inference-gbdt-with-lags-singlemodel
- EDA(1) : https://www.kaggle.com/code/motono0223/eda-jane-street-real-time-market-data-forecasting
- EDA(2) : https://www.kaggle.com/code/motono0223/eda-v2-jane-street-real-time-market-forecasting

In [1]:
import pandas as pd
import polars as pl
import numpy as np
import os
from tqdm.auto import tqdm
from matplotlib import pyplot as plt
import pickle

from sklearn.metrics import r2_score
from lightgbm import LGBMRegressor
import lightgbm as lgb
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

import kaggle_evaluation.jane_street_inference_server

# Configurations

In [2]:
class CONFIG:
    seed = 42
    target_col = "responder_6"
    feature_cols = ["symbol_id", "time_id"] \
        + [f"feature_{idx:02d}" for idx in range(79)] \
        + [f"responder_{idx}_lag_1" for idx in range(9)]
    categorical_cols = []

# Load Data

In [3]:
train = pl.scan_parquet("/kaggle/input/js24-preprocessing-create-lags/training.parquet").collect().to_pandas()
valid = pl.scan_parquet("/kaggle/input/js24-preprocessing-create-lags/validation.parquet").collect().to_pandas()
train.shape, valid.shape

((21022056, 104), (1082224, 104))

scan_parquet はデータをすぐにメモリに読み込むのではなく、後で処理するためのクエリプランを作成します（遅延評価）。

.collect(): 遅延評価を実行し、実際にデータをメモリに読み込みます。これによって、Polars の DataFrame が作成されます。

.to_pandas(): Polars の DataFrame を Pandas の DataFrame に変換します。これは、多くの機械学習ライブラリが Pandas の DataFrame を入力として受け取るためによく行われる処理です。

In [6]:
train

,id,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8,partition_id,label,responder_0_lag_1,responder_1_lag_1,responder_2_lag_1,responder_3_lag_1,responder_4_lag_1,responder_5_lag_1,responder_6_lag_1,responder_7_lag_1,responder_8_lag_1
0,25023058,1101,0,0,1.913215,0.458568,-0.052640,-0.044876,0.124223,2.819707,-0.746395,0.270162,0.232489,0.638483,11,7,76,-0.859027,0.695877,-0.356197,NaN,0.300569,NaN,-1.081869,-1.249594,0.586737,-0.283532,0.351779,1.073560,-0.974062,-0.908026,0.737549,0.875670,0.616378,0.052371,0.482981,-0.269024,NaN,NaN,-0.101967,-0.305440,0.745993,0.124049,0.335976,NaN,0.019273,NaN,NaN,-0.228946,NaN,-1.508707,0.995993,-1.614079,-0.025769,0.214689,NaN,1.344184,NaN,NaN,-1.083561,NaN,-1.226218,1.564233,NaN,0.315116,0.102745,0.659961,-0.140479,0.042491,-0.028724,-1.427209,-1.993070,-0.862814,1.568264,-0.094396,-0.809437,-0.145835,-0.684791,NaN,NaN,-0.122062,-0.378493,-0.364320,-0.260619,0.108411,-0.189889,0.095152,1.188086,0.183916,-0.569924,1.233719,0.429217,-0.708557,6,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,25023059,1101,0,1,4.360143,0.111398,0.500473,0.083353,0.395555,2.468412,-0.706918,0.313437,0.172877,0.371777,11,7,76,-0.878114,1.199868,-0.304281,NaN,-0.151362,NaN,-1.775065,-1.312861,0.167220,0.034493,2.383581,1.702920,-0.743340,-0.219267,0.377675,1.223783,1.276546,-0.343797,-0.154821,0.051061,NaN,NaN,-0.533578,-0.531406,1.022045,-0.023691,0.231311,NaN,0.783220,NaN,NaN,-0.969126,NaN,-1.442822,1.633847,-0.647865,0.110672,0.106416,NaN,1.489494,NaN,NaN,-0.510090,NaN,-0.935670,1.509301,NaN,0.675541,0.269578,0.659961,-0.277945,-0.191327,-0.312494,-1.107761,-1.085603,-0.759455,1.070539,-0.126439,-1.209990,0.309990,-0.537283,NaN,NaN,-0.223740,-0.256861,-0.288086,-0.323778,-0.155402,-0.154787,-0.243248,-0.497277,-0.258040,-0.358548,-0.582184,-0.333726,-0.268150,6,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,25023060,1101,0,2,1.259379,0.536855,0.309993,0.015098,0.318689,2.594687,-1.157400,0.485360,0.329561,0.671811,81,2,59,-1.163686,-0.307989,-0.858999,NaN,-0.307050,NaN,-1.227922,-1.326199,-1.473596,-0.266024,-0.474967,-0.199162,-1.760276,-1.485319,0.677112,1.269566,1.747018,-0.725915,-0.495244,-0.274749,NaN,NaN,-0.027293,-0.618841,0.878567,0.034158,0.406216,NaN,0.084095,NaN,NaN,-1.285599,NaN,-1.227116,0.667309,-0.322078,0.078113,0.506323,NaN,-0.006689,NaN,NaN,0.489376,NaN,-1.457590,2.773031,NaN,0.681339,0.411752,0.659961,-0.280476,-0.211912,-0.219207,-2.506536,-2.055502,-1.226109,-0.096572,-0.445255,-1.033872,-0.290695,-0.800315,NaN,NaN,-0.104352,-0.221018,-0.382566,-0.330158,-0.327308,-0.253688,-0.561014,-1.515344,1.919260,-3.469981,-1.465263,1.964456,-3.273642,6,-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,25023061,1101,0,3,1.498643,-0.016845,-0.104391,-0.202445,0.404921,2.384175,-0.447801,0.226170,0.172059,0.363244,4,3,11,-0.915372,1.030181,-0.212352,NaN,-0.169328,NaN,-1.573598,-1.306565,-0.405128,0.050558,-0.176753,-0.542888,0.304873,1.171678,-0.502733,-1.080110,-0.591162,-0.527882,-0.743914,0.045586,NaN,NaN,-0.380765,-0.613219,-0.657111,0.168769,0.181173,NaN,-0.130411,NaN,NaN,

In [ ]:
# Trick of boosting LB score: 0.45->0.49
train = pd.concat([train, valid]).reset_index(drop=True)
train.shape

トレーニングデータ (train) とバリデーションデータ (valid) を結合し、インデックスをリセットすることで、一つの大きなトレーニングデータを作成

pd.concat([train, valid]): Pandas の concat 関数を使用して、train と valid の DataFrame を縦方向に結合します。

[train, valid]：結合する DataFrame のリストです。リストの順に上から下に結合されます。
デフォルトでは、concat はインデックスを維持します。つまり、元の train と valid のインデックスがそのまま結合後の DataFrame に引き継がれます。
.reset_index(drop=True): 結合後の DataFrame のインデックスをリセットします。

reset_index(): インデックスをリセットする関数です。
drop=True: 元のインデックスを新しい列として追加するのではなく、完全に削除します。これにより、0 から始まる連続した整数の新しいインデックスが作成されます。

# GBDT models

In [1]:
def get_model(seed):
    # XGBoost parameters
    XGB_Params = {
        'learning_rate': 0.05,
        'max_depth': 6,
        'n_estimators': 200,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'reg_alpha': 1,
        'reg_lambda': 5,
        'random_state': seed,
        'tree_method': 'gpu_hist',
        'device' : 'cuda',
        'n_gpus' : 2,
    }
    
    XGB_Model = XGBRegressor(**XGB_Params)
    return XGB_Model

このコードは、XGBoost の回帰モデル (XGBRegressor) を生成する関数 get_model(seed) を定義しています。この関数は乱数シード (seed) を引数に取り、特定のパラメータ設定で XGBoost モデルを初期化して返します。

# Training model

In [ ]:
X_train = train[ CONFIG.feature_cols ]
y_train = train[ CONFIG.target_col ]
w_train = train[ "weight" ]
X_valid = valid[ CONFIG.feature_cols ]
y_valid = valid[ CONFIG.target_col ]
w_valid = valid[ "weight" ]

X_train.shape, y_train.shape, w_train.shape, X_valid.shape, y_valid.shape, w_valid.shape

w_train = train["weight"]: トレーニングデータ train から、重みの列を抽出します。

"weight": 重みの列名です。
train["weight"]: train DataFrame から "weight" 列を抽出します。抽出されたデータは Pandas の Series として w_train に格納されます。重みは、各データポイントの重要度を表し、損失関数などで使用されます。

In [ ]:
%%time
model = get_model(CONFIG.seed)
model.fit( X_train, y_train, sample_weight=w_train)

このコードは、get_model 関数で生成した XGBoost モデル (model) を、トレーニングデータ (X_train, y_train) と重み (w_train) を使って学習させています。そして、処理にかかった時間を計測しています

sample_weight=w_train: 各トレーニングデータポイントの重みを指定します。w_train は各データポイントに対応する重みの配列です。重みを与えることで、特定のデータポイントを重視した学習を行うことができます。例えば、重要なデータや、誤分類した場合の損失が大きいデータに対して大きな重みを与えることで、モデルの性能を向上させることが期待できます。もしsample_weightを指定しない場合は、全てのデータポイントに対して等しい重みが与えられます。


In [ ]:
y_pred_train1 = model.predict(X_train.iloc[:X_train.shape[0]//2])
y_pred_train2 = model.predict(X_train.iloc[X_train.shape[0]//2:])
train_score = r2_score(y_train, np.concatenate([y_pred_train1, y_pred_train2], axis=0), sample_weight=w_train )
train_score

このコードは、学習済みモデル (model) を使ってトレーニングデータの予測を行い、その予測結果を評価しています。特に、トレーニングデータを前半と後半に分割して予測を行い、最後にそれらを結合して全体のスコアを計算している点が特徴です。

y_pred_train1 = model.predict(X_train.iloc[:X_train.shape[0]//2]): トレーニングデータ X_train の前半部分を使って予測を行います。

X_train.iloc[:X_train.shape[0]//2]: X_train の最初の行から、X_train の行数の半分までの行を選択します。X_train.shape[0] は X_train の行数を表し、//2 は整数除算（小数点以下切り捨て）を行います。これにより、X_train をほぼ半分に分割します。.iloc は行番号でアクセスするためのインデクサです。
model.predict(...): 学習済みモデル model を使って予測を行います。入力として X_train の前半部分を渡しています。予測結果は y_pred_train1 に格納されます。

np.concatenate([y_pred_train1, y_pred_train2], axis=0): 前半部分の予測結果 y_pred_train1 と後半部分の予測結果 y_pred_train2 を結合します。axis=0 は縦方向（行方向）に結合することを意味します。これにより、X_train 全体に対する予測結果が得られます。

r2_score(y_train, ..., sample_weight=w_train): R2 スコア（決定係数）を計算します。
y_train: 正解のターゲット変数です。
np.concatenate(...): 予測されたターゲット変数です。
sample_weight=w_train: 各データポイントの重みを指定します。これにより、重みを考慮した R2 スコアが計算されます。

このコードの目的:

このコードの主な目的は、トレーニングデータ全体に対するモデルの性能を評価することです。トレーニングデータを分割して予測を行う理由は、メモリ使用量を抑えるため、または何らかの理由で一度に全てのデータを処理できない場合に有効です。特に大きなデータセットを扱う場合に有用です。

なぜ分割して予測するのか？

大きなデータセットの場合、一度に model.predict(X_train) を実行すると、メモリ不足になる可能性があります。そのため、データを分割して予測を行い、最後に結果を結合することで、メモリ消費を抑えながら予測を行うことができます。

In [ ]:
y_pred_valid = model.predict(X_valid)
valid_score = r2_score(y_valid, y_pred_valid, sample_weight=w_valid )
valid_score

In [ ]:
y_means = { symbol_id : -1 for symbol_id in range(39) }
for symbol_id, gdf in train[["symbol_id", CONFIG.target_col]].groupby("symbol_id"):
    y_mean = gdf[ CONFIG.target_col ].mean()
    y_means[symbol_id] = y_mean
    print(f"symbol_id = {symbol_id}, y_means = {y_mean:.5f}")

このコードは、トレーニングデータ train を symbol_id ごとにグループ化し、各 symbol_id におけるターゲット変数 (CONFIG.target_col) の平均値を計算しています。そして、その結果を辞書 y_means に格納しています。

-----


y_means = { symbol_id : -1 for symbol_id in range(39) }: y_means という辞書を初期化しています。

{ symbol_id : -1 for symbol_id in range(39) }: 辞書内包表記を使って、キーが 0 から 38 までの symbol_id、値が -1 の辞書を作成します。これは、各 symbol_id の平均値を格納するための準備で、初期値として -1 を設定しています。もし後で平均値が計算されなかった symbol_id があれば、その値は -1 のままになります。

for symbol_id, gdf in train[["symbol_id", CONFIG.target_col]].groupby("symbol_id"):: train データフレームを symbol_id でグループ化し、各グループに対して処理を行います。

train[["symbol_id", CONFIG.target_col]]: train から symbol_id 列と CONFIG.target_col 列のみを選択します。これにより、必要な列だけを処理することで効率を高めます。
.groupby("symbol_id"): 選択されたデータフレームを symbol_id でグループ化します。
for symbol_id, gdf in ...: グループ化された結果をループで処理します。symbol_id には現在のグループの symbol_id の値が、gdf にはその symbol_id に対応するデータフレーム（グループ）が格納されます。
y_mean = gdf[CONFIG.target_col].mean(): 現在のグループ (gdf) における CONFIG.target_col の平均値を計算します。

gdf[CONFIG.target_col]: 現在のグループの CONFIG.target_col 列を選択します。
.mean(): 選択された列の平均値を計算します。
計算された平均値は y_mean に格納されます。
y_means[symbol_id] = y_mean: 計算された平均値 y_mean を y_means 辞書に格納します。キーは現在の symbol_id、値は計算された平均値です。

print(f"symbol_id = {symbol_id}, y_means = {y_mean:.5f}"): 現在の symbol_id と計算された平均値を表示します。.5f は小数点以下 5 桁まで表示するフォーマット指定子です。

このコードの目的:

このコードの目的は、各 symbol_id ごとのターゲット変数の平均値を計算し、後で利用できるように y_means 辞書に格納することです。この平均値は、例えば予測値の補正や、特徴量として使用するなど、様々な用途に利用できます。

まとめ:

このコードは、train データフレームを symbol_id でグループ化し、各グループの CONFIG.target_col の平均値を計算して y_means 辞書に格納し、結果を表示します。初期化時に-1を設定することで、データに存在しないsymbol_idを区別できます。

In [ ]:
cv_detail = { symbol_id : 0 for symbol_id in range(39) }
for symbol_id, gdf in valid.groupby("symbol_id"):
    X_valid = gdf[ CONFIG.feature_cols ]
    y_valid = gdf[ CONFIG.target_col ]
    w_valid = gdf[ "weight" ]
    y_pred_valid = model.predict(X_valid)
    score = r2_score(y_valid, y_pred_valid, sample_weight=w_valid )
    cv_detail[symbol_id] = score
    
    print(f"symbol_id = {symbol_id}, score = {score:.5f}")

このコードは、バリデーションデータ valid を symbol_id ごとにグループ化し、各 symbol_id に対するモデルの性能（R2 スコア）を計算しています。そして、その結果を辞書 cv_detail に格納しています。詳しく解説します。

cv_detail = { symbol_id : 0 for symbol_id in range(39) }: cv_detail という辞書を初期化しています。

{ symbol_id : 0 for symbol_id in range(39) }: 辞書内包表記を使って、キーが 0 から 38 までの symbol_id、値が 0 の辞書を作成します。これは、各 symbol_id のスコアを格納するための準備で、初期値として 0 を設定しています。
for symbol_id, gdf in valid.groupby("symbol_id"):: バリデーションデータ valid を symbol_id でグループ化し、各グループに対して処理を行います。

valid.groupby("symbol_id"): valid DataFrame を symbol_id でグループ化します。
for symbol_id, gdf in ...: グループ化された結果をループで処理します。symbol_id には現在のグループの symbol_id の値が、gdf にはその symbol_id に対応するデータフレーム（グループ）が格納されます。
X_valid = gdf[CONFIG.feature_cols]: 現在のグループ (gdf) から特徴量 (CONFIG.feature_cols) を抽出します。

y_valid = gdf[CONFIG.target_col]: 現在のグループ (gdf) からターゲット変数 (CONFIG.target_col) を抽出します。

w_valid = gdf["weight"]: 現在のグループ (gdf) から重み ("weight") を抽出します。

y_pred_valid = model.predict(X_valid): 学習済みモデル (model) を使って、現在のグループのデータ (X_valid) に対する予測を行います。予測結果は y_pred_valid に格納されます。

score = r2_score(y_valid, y_pred_valid, sample_weight=w_valid): 現在のグループの予測結果を評価し、R2 スコアを計算します。

r2_score(y_valid, y_pred_valid, sample_weight=w_valid): R2 スコアを計算します。sample_weight を指定することで、重みを考慮したスコアが計算されます。
cv_detail[symbol_id] = score: 計算されたスコア (score) を cv_detail 辞書に格納します。キーは現在の symbol_id です。

print(f"symbol_id = {symbol_id}, score = {score:.5f}"): 現在の symbol_id と計算されたスコアを表示します。.5f は小数点以下 5 桁まで表示するフォーマット指定子です。

このコードの目的:

このコードの目的は、各 symbol_id ごとにモデルの性能を評価し、どの symbol_id でモデルの予測が良く、どの symbol_id で悪いのかを把握することです。これにより、モデルの改善点や、特定の symbol_id に特化したモデルの作成などを検討する材料となります。

まとめ:

このコードは、バリデーションデータを symbol_id でグループ化し、各グループに対して予測を行い、R2 スコアを計算して cv_detail 辞書に格納し、結果を表示します。これにより、symbol_id ごとのモデルの性能を詳細に分析できます。

In [ ]:
sids = list(cv_detail.keys())
plt.bar(sids, [cv_detail[sid] for sid in sids])
plt.grid()
plt.xlabel("symbol_id")
plt.ylabel("CV score")
plt.show()

sids = list(cv_detail.keys()): cv_detail 辞書のキー（symbol_id のリスト）を取得し、sids に格納します。

cv_detail.keys(): cv_detail 辞書のキー（symbol_id）のビューオブジェクトを返します。
list(...): ビューオブジェクトをリストに変換します。これにより、sids は symbol_id のリストになります。例えば、[0, 1, 2, ..., 38] のようになります。
plt.bar(sids, [cv_detail[sid] for sid in sids]): 棒グラフを作成します。

plt.bar(x, height): 棒グラフを作成する matplotlib の関数です。
x: 各棒の x 座標（ここでは symbol_id）。
height: 各棒の高さ（ここでは cv_detail の値、つまり CV スコア）。
[cv_detail[sid] for sid in sids]: リスト内包表記を使って、各 symbol_id に対応する cv_detail の値（CV スコア）のリストを作成します。例えば、[0.5, 0.8, 0.7, ..., 0.6] のようになります。
plt.grid(): グラフにグリッド線を追加します。これにより、値の読み取りが容易になります。

このコードの目的:

このコードの目的は、各 symbol_id におけるモデルの性能（CV スコア）を視覚的に比較することです。棒グラフによって、どの symbol_id でスコアが高く、どの symbol_id でスコアが低いのかが一目でわかります。これにより、モデルの改善点を分析したり、特定の symbol_id に特化した対応を検討したりするのに役立ちます。

まとめ:

このコードは、cv_detail 辞書に格納された各 symbol_id の CV スコアを棒グラフで可視化します。plt.bar() で棒グラフを作成し、plt.grid() でグリッド線を追加、plt.xlabel() と plt.ylabel() で軸ラベルを設定、plt.show() でグラフを表示します。これにより、各 symbol_id におけるモデルの性能を視覚的に比較することができます。


# Save result

In [ ]:
result = {
    "model" : model,
    "cv" : valid_score,
    "cv_detail" : cv_detail,
    "y_mean" : y_means,
}
with open("result.pkl", "wb") as fp:
    pickle.dump(result, fp)

このコードは、学習済みモデル (model)、交差検証スコア (valid_score)、交差検証の詳細 (cv_detail)、およびターゲット変数の平均値 (y_means) をまとめて result という辞書に格納し、その辞書を pickle を使ってファイル result.pkl に保存しています。

result = { ... }: データを格納するための辞書 result を作成します。

"model": model: キー "model" に学習済みモデル model を格納します。
"cv": valid_score: キー "cv" に交差検証スコア valid_score を格納します。
"cv_detail": cv_detail: キー "cv_detail" に各 symbol_id の交差検証スコアの詳細 cv_detail を格納します。
"y_mean": y_means: キー "y_mean" に各 symbol_id のターゲット変数の平均値 y_means を格納します。
with open("result.pkl", "wb") as fp:: ファイル result.pkl をバイナリ書き込みモード ("wb") で開きます。

with ... as ...: with 文を使うことで、ファイル操作後のクローズ処理を自動的に行うことができます。これにより、ファイルの閉じ忘れによるエラーを防ぐことができます。
"result.pkl": 保存するファイル名です。拡張子 .pkl は pickle で保存されたファイルであることを示します。
"wb": ファイルをバイナリ書き込みモードで開きます。pickle はオブジェクトをバイナリ形式で保存するため、このモードで開く必要があります。
as fp: 開いたファイルオブジェクトを fp という変数に割り当てます。
pickle.dump(result, fp): result 辞書をファイル fp に保存します。

pickle.dump(obj, file): pickle モジュールの dump 関数を使って、オブジェクト (obj) をファイル (file) にシリアライズして保存します。シリアライズとは、Python オブジェクトをバイトストリームに変換して保存可能な形式にすることです。
result: 保存する辞書オブジェクトです。
fp: 書き込み先のファイルオブジェクトです。
このコードの目的:

このコードの目的は、学習済みモデルや評価結果などの重要なデータをファイルに保存し、後でロードして利用できるようにすることです。特に、学習に時間がかかるモデルの場合、学習済みのモデルを保存しておくことで、再度学習を行う必要がなくなり、時間と計算資源の節約になります。また、評価結果などを保存しておくことで、実験結果の再現や比較が容易になります。

pickle とは:

pickle は Python の標準ライブラリで、Python オブジェクトをシリアライズ（直列化）およびデシリアライズ（非直列化）するためのモジュールです。シリアライズとは、オブジェクトをバイトストリームに変換して保存可能な形式にすることです。デシリアライズとは、バイトストリームから元のオブジェクトを復元することです。

なぜバイナリモード ("wb") で開くのか:

pickle はオブジェクトをバイナリ形式で保存するため、ファイルをバイナリモードで開く必要があります。テキストモード ("w") で開くと、データが正しく保存されません。

とめ:

このコードは、学習済みモデル、評価結果などの重要なデータを result 辞書にまとめ、pickle.dump() を使って result.pkl ファイルに保存します。これにより、データを永続化し、後でロードして利用することができます。pickle はオブジェクトをバイナリ形式で保存するため、ファイルはバイナリモードで開く必要があります。